# 1. LAST-FM: Time Weighted ALS Recommender

This notebook implements a simple ALS recommender based on the LastFM user listening dataset. It uses spark and is written in Scala. Minimal data cleaning/pre-processing is performed to provide a baseline model. 

## 1.1 Imports and set up 

Key libraries are imported, the spark session is initialised and the listening data is loaded in. 

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.0.0` // Or use any other 2.x version here

import $ivy.$                                   // Or use any other 2.x version here

In [2]:
import $ivy.`org.apache.spark::spark-mllib:3.0.0`

import $ivy.$                                    

In [3]:
import $ivy.`org.vegas-viz:vegas_2.11:0.3.11`
import $ivy.`org.vegas-viz:vegas-spark_2.11:0.3.11`

Downloaded https://repo1.maven.org/maven2/org/vegas-viz/vegas_2.11/0.3.11/vegas_2.11-0.3.11.pom
Downloaded https://repo1.maven.org/maven2/com/github/julien-truffaut/monocle-macro_2.11/1.1.0/monocle-macro_2.11-1.1.0.pom
Downloaded https://repo1.maven.org/maven2/io/circe/circe-generic_2.11/0.7.0/circe-generic_2.11-0.7.0.pom
Downloaded https://repo1.maven.org/maven2/org/vegas-viz/vegas-macros_2.11/0.3.11/vegas-macros_2.11-0.3.11.pom
Downloaded https://repo1.maven.org/maven2/io/circe/circe-parser_2.11/0.7.0/circe-parser_2.11-0.7.0.pom
Downloaded https://repo1.maven.org/maven2/org/scalafx/scalafx_2.11/8.0.92-R10/scalafx_2.11-8.0.92-R10.pom
Downloaded https://repo1.maven.org/maven2/com/github/julien-truffaut/monocle-core_2.11/1.1.0/monocle-core_2.11-1.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/webjars/bower/vega-lite/1.2.0/vega-lite-1.2.0.pom
Downloaded https://repo1.maven.org/maven2/io/circe/circe-core_2.11/0.7.0/circe-core_2.11-0.7.0.pom
Downloaded https://repo1.maven.org/maven2

Downloaded https://repo1.maven.org/maven2/org/typelevel/cats-macros_2.11/0.9.0/cats-macros_2.11-0.9.0-sources.jar
Downloaded https://repo1.maven.org/maven2/io/circe/circe-parser_2.11/0.7.0/circe-parser_2.11-0.7.0.jar
Downloaded https://repo1.maven.org/maven2/org/spire-math/jawn-parser_2.11/0.10.4/jawn-parser_2.11-0.10.4.jar
Downloaded https://repo1.maven.org/maven2/org/vegas-viz/vegas-macros_2.11/0.3.11/vegas-macros_2.11-0.3.11.jar
Downloaded https://repo1.maven.org/maven2/org/scalafx/scalafx_2.11/8.0.92-R10/scalafx_2.11-8.0.92-R10.jar
Downloaded https://repo1.maven.org/maven2/org/spire-math/jawn-parser_2.11/0.10.4/jawn-parser_2.11-0.10.4-sources.jar
Downloaded https://repo1.maven.org/maven2/org/typelevel/cats-macros_2.11/0.9.0/cats-macros_2.11-0.9.0.jar
Downloaded https://repo1.maven.org/maven2/io/circe/circe-core_2.11/0.7.0/circe-core_2.11-0.7.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/webjars/bower/vega-lite/1.2.0/vega-lite-1.2.0.jar
Downloaded https://repo1.maven.o

import $ivy.$                                

import $ivy.$                                      

In [4]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.{StringIndexer,IndexToString}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.mllib.evaluation.{RankingMetrics, RegressionMetrics}
import org.apache.spark.ml.feature.RobustScaler
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.functions.vector_to_array
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
import vegas._
import vegas.render.WindowRenderer._
import vegas.sparkExt._

import org.apache.spark.sql.SparkSession

import org.apache.spark.ml.feature.{StringIndexer,IndexToString}

import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.recommendation.ALS

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.mllib.evaluation.{RankingMetrics, RegressionMetrics}

import org.apache.spark.ml.feature.RobustScaler

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.functions.vector_to_array

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql._

import vegas._

import vegas.render.WindowRenderer._

import vegas.sparkExt._

In [ ]:
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

In [5]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)



val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .appName("lastfm")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.log4j.{Level, Logger}

spark: SparkSession = org.apache.spark.sql.SparkSession@f835705

In [6]:
// path to last-fm dataset. Can be downloaded here: http://millionsongdataset.com/lastfm/
var data_path:String = "../resources/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

data_path: String = "../resources/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

In [7]:
// schema defined below to set column names and types. 
val schema = new StructType()
            .add("user_id", StringType, true)
            .add("timestamp", DateType, true)
            .add("artist_id", StringType, true)
            .add("artist_name", StringType, true)
            .add("track_id", StringType, true)
            .add("track_name", StringType, true)

schema: StructType = StructType(
  StructField("user_id", StringType, true, {}),
  StructField("timestamp", DateType, true, {}),
  StructField("artist_id", StringType, true, {}),
  StructField("artist_name", StringType, true, {}),
  StructField("track_id", StringType, true, {}),
  StructField("track_name", StringType, true, {})
)

In [8]:
// read in data
val listener_data = spark
                    .read
                    .options(Map(
                      "header" -> "false",
                      "dateFormat" -> "yyyy-MM-ddTHH:mm:ssZ"
                    ))
                    .schema(schema)
                    .format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat")
                    .option("sep", "\t")
                    .load(data_path)
listener_data.show()

show at cmd7.sc:11

1 / 1

+-----------+----------+--------------------+---------------+--------------------+--------------------+
|    user_id| timestamp|           artist_id|    artist_name|            track_id|          track_name|
+-----------+----------+--------------------+---------------+--------------------+--------------------+
|user_000001|2009-05-04|f1b1cf71-bd35-4e9...|      Deep Dish|                null|Fuck Me Im Famous...|
|user_000001|2009-05-04|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Composition 0919 ...|
|user_000001|2009-05-04|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Mc2 (Live_2009_4_15)|
|user_000001|2009-05-04|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Hibari (Live_2009...|
|user_000001|2009-05-04|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Mc1 (Live_2009_4_15)|
|user_000001|2009-05-04|a7f7df4a-77d8-4f1...|       坂本龍一|                null|To Stanford (Live...|
|user_000001|2009-05-04|a7f7df4a-77d8-4f1...|       坂本龍一|                null|Improv

listener_data: DataFrame = [user_id: string, timestamp: date ... 4 more fields]

In [9]:
import spark.implicits._

val df = listener_data.na.drop()
val max_date = df.agg(max("timestamp")).map(_.getDate(0)).collectAsList().toArray()(0)

collectAsList at cmd8.sc:4

19 / 19

collectAsList at cmd8.sc:4

1 / 1

import spark.implicits._


df: DataFrame = [user_id: string, timestamp: date ... 4 more fields]
max_date: Object = 2013-09-29

In [10]:
val date_time_df = df.select("user_id", "track_id", "timestamp")
        .groupBy("user_id", "track_id")
        .agg(count("*").alias("count"), max("timestamp").alias("last_heard")).withColumn("max_date", lit(max_date))
val df_agg = date_time_df
             .withColumn("datediff", datediff(col("max_date"), col("last_heard")))
             .select("user_id", "track_id", "count", "datediff")
             .orderBy("datediff")
val df_agg_filtered = df_agg.limit(5000)
df_agg_filtered.show()

show at cmd9.sc:9

19 / 19

show at cmd9.sc:9

200 / 200

show at cmd9.sc:9

1 / 1

+-----------+--------------------+-----+--------+
|    user_id|            track_id|count|datediff|
+-----------+--------------------+-----+--------+
|user_000762|b5b40605-5a81-46b...|    1|       0|
|user_000405|223f9324-3546-446...|    1|    1143|
|user_000310|d8c5d65d-add9-492...|    1|    1563|
|user_000651|db44f801-19de-43b...|    6|    1563|
|user_000724|4d3dffd2-d8bb-4a4...|    9|    1563|
|user_000612|459b467a-81f1-4a0...|   26|    1563|
|user_000021|681eab7c-75fb-400...|    7|    1563|
|user_000724|22a4c37b-dbac-465...|   12|    1563|
|user_000960|4c7e938d-c4e2-48b...|   10|    1563|
|user_000960|fe1e042b-e443-4f3...|   32|    1563|
|user_000612|b0990649-efb6-469...|  198|    1563|
|user_000960|bb7770b2-67e2-491...|   68|    1563|
|user_000651|b1c5184c-70c8-4fa...|    8|    1563|
|user_000651|d02aa861-ac38-418...|   26|    1563|
|user_000651|bfba20d5-7995-436...|   30|    1563|
|user_000651|3ffed2aa-435d-41f...|   22|    1563|
|user_000651|d9f41e96-e7c5-401...|   19|    1563|


date_time_df: DataFrame = [user_id: string, track_id: string ... 3 more fields]
df_agg: Dataset[Row] = [user_id: string, track_id: string ... 2 more fields]
df_agg_filtered: Dataset[Row] = [user_id: string, track_id: string ... 2 more fields]

In [12]:
// remove outliers
// do weighting by date

Vegas(“Aggregated LastFM Distribution”)
 .withDataFrame(df_agg_filtered)
 .mark(Bar) 
 .encodeX(“language”, Nom, sortField=Sort(“users_count”, AggOps.Mean))
 .encodeY(“users_count”, Quant)
 .show

take at cmd10.sc:4

19 / 19

take at cmd10.sc:4

200 / 200

user_id,track_id,count,datediff
user_000762,b5b40605-5a81-46b4-a51e-2b1ec7964c1a,1,0
user_000405,223f9324-3546-446b-96a2-5f663192dd42,1,1143
user_000651,d9f41e96-e7c5-401b-ae84-a2032a631409,19,1563
user_000960,23c221b8-faac-426f-939c-ca3a016533dd,8,1563
user_000651,b1c5184c-70c8-4fac-83e1-bb81af50028b,8,1563
user_000960,62b35771-1db8-48e3-83be-6683729e3e82,3,1563
user_000651,22337e71-0bf8-4fa9-86eb-2f8e7e085474,31,1563
user_000651,8a7d7cce-454a-4d68-9980-8fa5cb03cad9,14,1563
user_000612,86f07bfb-eb6c-418b-9825-ee8e909e799d,29,1563
user_000612,fe976c7e-1f92-4d47-bc0d-aa4f26e4c0f0,44,1563


The dataframe is then aggregated by user and track, to get the number of times a user has heard a particular track. 

In [ ]:
val Array(training, test) = df_agg_filtered.randomSplit(Array[Double](0.8, 0.2), 18)

//revisit to make more efficient

val feat = df_agg_filtered.columns.filter(_ .contains("id"))
val inds = feat.map { colName =>
   new StringIndexer()
    .setInputCol(colName)
    .setOutputCol(colName.replace("id", "index"))
    .fit(df_agg_filtered)
    .setHandleInvalid("keep")
}

val va = new VectorAssembler()
    .setInputCols(Array("count"))
    .setOutputCol("count_assembled")

val scaler = new RobustScaler()
  .setInputCol("count_assembled")
  .setOutputCol("rating")

val pipeline = new Pipeline()
  .setStages(inds.toArray ++ Array(va, scaler))
  
val tr_s = pipeline.fit(training).transform(training)
val ts_s = pipeline.fit(training).transform(test)

val tr_full = tr_s.withColumn("rating_as_array", vector_to_array(tr_s("rating")).getItem(0))
val ts_full = ts_s.withColumn("rating_as_array", vector_to_array(ts_s("rating")).getItem(0))

val tr_final = tr_full.select("user_index", "track_index", "count","rating_as_array").orderBy("user_index")
val ts_final = ts_full.select("user_index", "track_index", "count", "rating_as_array").orderBy("user_index")

In [ ]:
tr_final.show()

In [ ]:
val als = new ALS()
  .setRank(5)
  .setUserCol("user_index")
  .setItemCol("track_index")
  .setRatingCol("rating_as_array")

val model = als.fit(tr_final)
model.setColdStartStrategy("drop")

val predictions = model.transform(ts_final)

val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating_as_array")
  .setPredictionCol("prediction")

val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")


In [ ]:
val userRecs = model.recommendForAllUsers(10)
val movieRecs = model.recommendForAllItems(10)

In [ ]:
val firsttworecs = userRecs
    .withColumn("columns",expr("struct(recommendations[0] as rec1, recommendations[0] as rec2) as columns"))
    .select("user_index","columns.*")
firsttworecs.show()

In [ ]:
val firstRec = firsttworecs.select("user_index", "rec1.*")
firstRec.show()

In [ ]:
// TODO make more efficient 
import spark.implicits._
val users = df_agg_filtered.select("user_id").map(_.getString(0)).distinct().collectAsList().toArray().map(_.asInstanceOf[String])
val usermap = new IndexToString()
    .setInputCol("user_index")
    .setOutputCol("user_id")
    .setLabels(users)

val userout = usermap.transform(firstRec)

val tracks = df_agg_filtered.select("track_id").map(_.getString(0)).distinct().collectAsList().toArray().map(_.asInstanceOf[String])
val trackmap = new IndexToString()
    .setInputCol("track_index")
    .setOutputCol("track_id")
    .setLabels(tracks)

val trackout = trackmap.transform(userout)

In [ ]:
trackout.show()

In [ ]:
val final_result = trackout.as("results")
        .join(df.as("in"), $"results.track_id" === $"in.track_id")
        .select("results.user_id", "in.track_name", "results.rating").distinct()
final_result.show()